<h1 style="background-color:lightblue;font-family:newtimeroman;text-align:center;border-radius: 50px 0px"><font color=black size=6>Modelagem e Extração do Resultado da OAB</font></h1>

Passos:
1. retirar todos os caracteres antes dos números 1.1, 1.2...
2. Retirar todos os caracteres após o '.'. Obrigatoriamente deve ser rodado após o passo 1

[Site Resultado em pdf](https://oab.fgv.br/arq/642/2464093_Resultado%20definitivo%20da%201%C2%AA%20fase%20-%2039%C2%BA%20EOU.pdf)

## Bibliotecas

In [5]:
import pandas as pd
import numpy as np

import re

## Inputs

In [ ]:
qual_exame = input("Qual o exame da OAB? Ex: 37,38,39, etc:\n")
fase = input("Qual a fase do exame de ordem, 1 ou 2?:\n")
preencher_1_UF = input("\nQual a UF a ser preenchida na primeira cidade?\n")
texto = input("Cole aqui os dados - Todo o conteúdo do pdf:\n")


# Encontrar a primeira ocorrência de '1.1.'
match = re.search(r'1\.1\.', texto)

if match:
    indice = match.start()
    resultado = texto[indice:]
else:
    resultado = texto

##### RETIRAR DO TEXTO #####
# 1a fase #
texto_retirar = 'II - Relação dos examinandos aprovados na prova objetiva, na condição sub judice, na seguinte ordem: seccional, cidade de prova, número de inscrição e nome do examinando em ordem alfabética'
resultado = resultado.replace(texto_retirar, '')

# Encontrar a primeira ocorrência de 'DAS DISPOSIÇÕES FINAIS '
indice_final = resultado.find('2 DAS DISPOSIÇÕES FINAIS')

if indice_final != -1:  # Se a substring for encontrada
    resultado = resultado[:indice_final]  # Pegando a parte do texto até o índice onde 'DAS DISPOSIÇÕES FINAIS' começa
else:
    resultado = resultado

# 2a fase #
texto_retirar_2 = f'II - Relação de examinandos aprovados no {qual_exame}º Exame de Ordem Unificado, na condição sub judice, na seguinte ordem: seccional, cidade de prova, número de inscrição e nome do examinando em ordem alfabética.'
texto_retirar_3 = f'II - Relação de examinandos aprovados no {qual_exame}o Exame de Ordem Unificado, na condição sub judice, na seguinte ordem: seccional, cidade de prova, número de inscrição e nome do examinando em ordem alfabética.'
resultado = resultado.replace(texto_retirar_2, '')
resultado = resultado.replace(texto_retirar_3, '')



# Encontrar a primeira ocorrência de 'DAS DISPOSIÇÕES FINAIS '
indice_final = resultado.find('2 DISPOSIÇÕES FINAIS')

if indice_final != -1:  # Se a substring for encontrada
    resultado = resultado[:indice_final]  # Pegando a parte do texto até o índice onde 'DAS DISPOSIÇÕES FINAIS' começa
else:
    resultado = resultado


## Modelagem

#### Ajuste para captura dos nomes

In [3]:
# Colocando '/' antes de das cidades que não estão acompanhadas de UF
for i in range(1, 28):
    for j in range(2, 101):
        elemento = str(i) + '.' + str(j) + '.'
        resultado = resultado.replace(elemento, '/' + elemento)

lista_oab = resultado.split('/')

#### Criando o DF com a base

In [4]:
# Número de linhas (aprovados)
n_rows = len(lista_oab)

# Criando 2 listas: 1a Matrículas / 2a Nomes
list_rows = [lista_oab[i].split(',') for i in range(n_rows)]


In [5]:
# Criando o dataframe com as listas criadas
df_lista_oab = pd.DataFrame(list_rows, columns=['Matrícula','Nome'])
df_lista_oab = df_lista_oab[df_lista_oab['Matrícula'].notnull() & df_lista_oab['Nome'].notnull()]


#### Ajustando os campos de Matrúcula e UF/Cidade

In [6]:
df_lista_oab['Matrícula Ajustada'] = df_lista_oab['Matrícula'].str.slice(start=-9)
df_lista_oab['Cidade/UF'] = df_lista_oab['Matrícula'].str.slice(stop=-9)


# Retirando o possível '.' do nome
for i in range(df_lista_oab.shape[0]):
    df_lista_oab['Nome'].iloc[i] = df_lista_oab['Nome'].iloc[i].replace('.',' ')
    df_lista_oab['Nome'].iloc[i] = df_lista_oab['Nome'].iloc[i].replace('OAB',' ')
    df_lista_oab['Matrícula Ajustada'].iloc[i] = df_lista_oab['Matrícula Ajustada'].iloc[i].replace('.',' ')
    df_lista_oab['Matrícula Ajustada'].iloc[i] = df_lista_oab['Matrícula Ajustada'].iloc[i].replace('OAB',' ')

#### Removendo os espaços de Nome e Matrícula

In [7]:
##############
# Função para remover números precedidos e sucedidos por espaço em uma string
def remover_numeros_com_espaco(string):
    return re.sub(r'(?<=\s)\d+(?=\s)', '', string)

# Aplicando a função à coluna
df_lista_oab['Nome'] = df_lista_oab['Nome'].apply(remover_numeros_com_espaco)
df_lista_oab['Nome'] = df_lista_oab['Nome'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
df_lista_oab['Matrícula Ajustada'] = df_lista_oab['Matrícula Ajustada'].apply(remover_numeros_com_espaco)


#### Crindo campos UF e Cidade

In [8]:
# Pegando a Cidade e UF
df_lista_oab['UF'] = df_lista_oab['Cidade/UF']\
    .str.extract(r'^([^\d]+)')
df_lista_oab['Cidade'] = df_lista_oab['Cidade/UF']\
    .str.split('.').str.get(-1)\
    .str.strip()

# Dropando colunas desnecessárias
df_lista_oab = df_lista_oab.drop(['Matrícula','Cidade/UF'], axis=1)


In [9]:
df_lista_oab.head(3)

,Nome,Matrícula Ajustada,UF,Cidade
0,Alaissa Nascimento Galvão,395154248,NaN,RIO BRANCO
1,Alana Nascimento E Souza,395050404,,
2,Aleks Rodrigues Barboza Junior,395146723,,


#### Ajustando o 1º nome de UF --> Veio faltando para o RIO BRANCO

In [11]:
df_lista_oab.loc[0, 'UF'] = preencher_1_UF

#### Retirando os espaços antes e depois dos caracteres & upper nome

In [12]:
for i in df_lista_oab.columns:
    df_lista_oab[i] = df_lista_oab[i].str.strip()\
        .str.upper()


#### Ajustando UF e Cidade

In [13]:
# função para remover os números
def remover_numeros(valor):
    if pd.isna(valor):  # Verificar se o valor é nulo
        return valor  # Se for nulo, retornar o valor nulo
    else:
        return ''.join([i for i in valor if not i.isdigit()])  # Remover números do valor não nulo

# Aplicando a função
df_lista_oab['UF'] = df_lista_oab['UF'].apply(remover_numeros)
df_lista_oab['Cidade'] = df_lista_oab['Cidade'].apply(remover_numeros)

# Ajustando os nomes vazios em Cidade e UF
df_lista_oab['UF'] = [None 
    if (df_lista_oab['UF'].iloc[i] == '') or df_lista_oab['UF'].isnull().iloc[i] 
    else df_lista_oab['UF'].iloc[i] for i in range(df_lista_oab.shape[0])]

df_lista_oab['Cidade'] = [None 
    if (df_lista_oab['Cidade'].iloc[i] == '') or df_lista_oab['Cidade'].isnull().iloc[i] 
    else df_lista_oab['Cidade'].iloc[i] for i in range(df_lista_oab.shape[0])]

# Ajustando os valores vazios para os campos colocando os anteriores
df_lista_oab['UF'] = df_lista_oab.UF.str.strip().fillna(method='ffill')
df_lista_oab['Cidade'] = df_lista_oab.Cidade.str.strip().fillna(method='ffill')

#### Ajuste Final de espaço em branco

In [14]:
for i in range(df_lista_oab.shape[0]):
    df_lista_oab['Nome'].iloc[i] = df_lista_oab['Nome'].iloc[i].replace('  ',' ')
    df_lista_oab['Nome'].iloc[i] = df_lista_oab['Nome'].iloc[i].replace('   ',' ')
    df_lista_oab['Nome'].iloc[i] = df_lista_oab['Nome'].iloc[i].replace('    ',' ')
    df_lista_oab['Nome'].iloc[i] = df_lista_oab['Nome'].iloc[i].replace('     ',' ')
    df_lista_oab['Nome'].iloc[i] = df_lista_oab['Nome'].iloc[i].replace('      ',' ')

for i in range(df_lista_oab.shape[0]):
    df_lista_oab['Nome'].iloc[i] = df_lista_oab['Nome'].iloc[i].replace('  ',' ')
    df_lista_oab['Nome'].iloc[i] = df_lista_oab['Nome'].iloc[i].replace('   ',' ')
    df_lista_oab['Nome'].iloc[i] = df_lista_oab['Nome'].iloc[i].replace('    ',' ')
    df_lista_oab['Nome'].iloc[i] = df_lista_oab['Nome'].iloc[i].replace('     ',' ')
    df_lista_oab['Nome'].iloc[i] = df_lista_oab['Nome'].iloc[i].replace('      ',' ')

### Salvando a base final

In [15]:
df_lista_oab.to_csv(f"/Users/felipebarreto/Desktop/Dados Resultado OAB/Resultado {fase}a Fase {qual_exame}.csv")